<a href="https://colab.research.google.com/github/roudnel/Advanced_programming/blob/main/Rep_Dem_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trigger warning: This notebook contains words or language that are considered profane, vulgar, or offensive by some.

In [1]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis
!pip install --force-reinstall -q numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 77.2 MB/s eta 0:00:00


In [2]:
# explainability (why did the model say it's hate speech)
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 19.3 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=e75cabd8baa3ecf75b4307962f46129ae23720fca702b2bd0c61cf30e6611a08
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatib

In [5]:
import pandas as pd
import numpy as np
import preprocessor as prepro # twitter prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [6]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [7]:
data = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pol_tweets.gz')

In [8]:
data

,text,labels
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1
371088,Made new friends this afternoon delivering mas...,1
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0
476047,RT @SteveScalise President Trump's legal team ...,0
...,...,...
61499,Outrageous.\n\nBrave health care workers are p...,0
185562,RT @dskolnick .@RepTimRyan proposes up to $3K ...,1
354040,It is clear that the #HeroesAct will help tens...,1
708686,Democrats are talking about Bolton and Mulvane...,0


In [9]:
data['text_clean'] = data['text'].map(lambda t: prepro.clean(t))
data['text_clean'] = data['text_clean'].str.replace('#','')

In [10]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline

clean_text = []

pbar = tqdm.tqdm(total=len(data['text_clean']),position=0, leave=True)

for text in nlp.pipe(data['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|█████████▉| 49963/50000 [01:34<00:00, 647.18it/s]

In [11]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [12]:
# apply all prepro-pipeline to texts
data['text_clean'] = text_prepro(data['text'])

100%|██████████| 50000/50000 [01:29<00:00, 560.74it/s]


In [13]:
data

,text,labels,text_clean
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0,today remember sgt st class ryan savard killed...
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1,yes yes yes yes yes jerseystrong qt prioritize...
371088,Made new friends this afternoon delivering mas...,1,new friends afternoon delivering masks veteran...
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0,happening today pilots texas air guard conduct...
476047,RT @SteveScalise President Trump's legal team ...,0,president trump legal team completely destroye...
...,...,...,...
61499,Outrageous.\n\nBrave health care workers are p...,0,outrageous brave health care workers putting l...
185562,RT @dskolnick .@RepTimRyan proposes up to $3K ...,1,proposes k month family pandemic
354040,It is clear that the #HeroesAct will help tens...,1,clear heroesact help tens millions americans h...
708686,Democrats are talking about Bolton and Mulvane...,0,democrats talking bolton mulvaney witnesses co...


In [14]:
# renaming and reordering

data_df = pd.DataFrame({'label':data['labels'], 'text':data['text_clean']})

In [15]:

data_df.label.value_counts().reset_index()

,index,label
0,1,31181
1,0,18819


In [16]:
alt.Chart(data_df.label.value_counts().reset_index()).mark_bar(filled=True).encode(
    alt.X('label:Q', title='N Tweets'),
    alt.Y('index:N', title='Category')
)

alt.Chart(...)

In [17]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data_df, data_df['label'])

In [18]:
data_df_res['label'].value_counts()

0    18819
1    18819
Name: label, dtype: int64

In [19]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(data_df_res['text'], y_res, test_size = 0.4, random_state = 42)

In [20]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [21]:
pipe.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [22]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     11292
           1       0.87      0.84      0.85     11290

    accuracy                           0.86     22582
   macro avg       0.86      0.86      0.86     22582
weighted avg       0.86      0.86      0.86     22582



In [23]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=10, target_names=['Rep','Dem'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Topic model

In [24]:
# preprocess texts (we need tokens)
tokens = []

for text in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in text 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [25]:
data['tokens'] = tokens

In [26]:
data

,text,labels,text_clean,tokens
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0,today remember sgt st class ryan savard killed...,"[today, sgt, st, class, ryan, savard, action, ..."
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1,yes yes yes yes yes jerseystrong qt prioritize...,"[jerseystrong, science, politic, datum, politi..."
371088,Made new friends this afternoon delivering mas...,1,new friends afternoon delivering masks veteran...,"[new, friend, afternoon, mask, veteran, pete, ..."
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0,happening today pilots texas air guard conduct...,"[today, pilots, texas, air, guard, flyover, ho..."
476047,RT @SteveScalise President Trump's legal team ...,0,president trump legal team completely destroye...,"[president, trump, legal, team, completely, da..."
...,...,...,...,...
61499,Outrageous.\n\nBrave health care workers are p...,0,outrageous brave health care workers putting l...,"[outrageous, brave, health, care, worker, life..."
185562,RT @dskolnick .@RepTimRyan proposes up to $3K ...,1,proposes k month family pandemic,"[k, month, family, pandemic]"
354040,It is clear that the #HeroesAct will help tens...,1,clear heroesact help tens millions americans h...,"[clear, heroesact, ten, million, americans, se..."
708686,Democrats are talking about Bolton and Mulvane...,0,democrats talking bolton mulvaney witnesses co...,"[democrats, bolton, mulvaney, witness, course,..."


We would like to know what things people are talking about when it is considerede "hatespeech"

In [27]:
data_rep = data[data['labels'] == 0]

In [28]:
data_rep

,text,labels,text_clean,tokens
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0,today remember sgt st class ryan savard killed...,"[today, sgt, st, class, ryan, savard, action, ..."
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0,happening today pilots texas air guard conduct...,"[today, pilots, texas, air, guard, flyover, ho..."
476047,RT @SteveScalise President Trump's legal team ...,0,president trump legal team completely destroye...,"[president, trump, legal, team, completely, da..."
401277,Join me in watching this historic moment. #Lau...,0,join watching historic moment launchamerica qt...,"[historic, moment, launchamerica, welcome, lau..."
578814,As a proud pilot I have spent my career empowe...,0,proud pilot spent career empowering amp suppor...,"[proud, pilot, career, amp, aviation, aerospac..."
...,...,...,...,...
592618,Disappointed that Senate Dems blocked common-s...,0,disappointed senate dems blocked common sense ...,"[disappointed, senate, dem, common, sense, cov..."
168257,RT @JudiciaryGOP 🚨 #BREAKING: @Jim_Jordan blas...,0,breaking blasts nadler indicated intent subpoe...,"[blast, nadler, intent, subpoena, attorney, ge..."
401665,🚨🚨 https://twitter.com/WV_DHHR/status/12816386...,0,qt free testing held july barbour harrison mer...,"[free, testing, july, barbour, harrison, merce..."
61499,Outrageous.\n\nBrave health care workers are p...,0,outrageous brave health care workers putting l...,"[outrageous, brave, health, care, worker, life..."


In [30]:
from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data_rep['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data_rep['tokens']]

In [ ]:
corpus

[[(0, 1)],
 [(1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1)],
 [(7, 2), (8, 1), (9, 2)],
 [(8, 1), (10, 1)],
 [],
 [(11, 1), (12, 1), (13, 1)],
 [(8, 1), (14, 1)],
 [(15, 1)],
 [],
 [(16, 1)],
 [(17, 1)],
 [(17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 1)],
 [(8, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1)],
 [(8, 1), (29, 1), (30, 1)],
 [(15, 1), (22, 1), (31, 1)],
 [(9, 1), (30, 1), (32, 1), (33, 1), (34, 1)],
 [(17, 1), (20, 1), (35, 1)],
 [(8, 1), (15, 1), (36, 1)],
 [(0, 1)],
 [(21, 1), (37, 1)],
 [(5, 1), (9, 1), (12, 1), (38, 1), (39, 1)],
 [(40, 1)],
 [(17, 1), (41, 1), (42, 1)],
 [(17, 1), (43, 1)],
 [(37, 1), (41, 1), (44, 1), (45, 1), (46, 1), (47, 1)],
 [(17, 1), (33, 1), (37, 1), (48, 1)],
 [(49, 1)],
 [(50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1)],
 [(57, 1), (58, 1), (59, 1)],
 [(57, 1), (58, 1), (59, 1)],
 [(57, 1), (60, 1)],
 [(57, 1), (58, 1)],
 [(9, 1), (57, 1), (58, 1)],
 [(57, 1), (58, 1)],
 

In [31]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=10, workers = 4, passes=10)

In [32]:
lda_model.print_topics()

[(0,
  '0.021*"state" + 0.020*"school" + 0.019*"county" + 0.017*"home" + 0.016*"student" + 0.016*"new" + 0.014*"health" + 0.014*"today" + 0.013*"community" + 0.012*"local"'),
 (1,
  '0.077*"business" + 0.056*"small" + 0.032*"program" + 0.027*"amp" + 0.019*"relief" + 0.017*"loan" + 0.017*"coronavirus" + 0.015*"ppp" + 0.015*"protection" + 0.014*"paycheck"'),
 (2,
  '0.030*"act" + 0.023*"great" + 0.023*"american" + 0.021*"right" + 0.021*"bill" + 0.021*"court" + 0.015*"judge" + 0.015*"senate" + 0.013*"today" + 0.013*"life"'),
 (3,
  '0.041*"law" + 0.041*"police" + 0.026*"officer" + 0.025*"enforcement" + 0.022*"time" + 0.021*"family" + 0.019*"amp" + 0.012*"de" + 0.012*"community" + 0.011*"big"'),
 (4,
  '0.029*"job" + 0.027*"democrats" + 0.026*"people" + 0.025*"pelosi" + 0.022*"american" + 0.019*"bill" + 0.018*"americans" + 0.018*"house" + 0.017*"economy" + 0.015*"congress"'),
 (5,
  '0.025*"vote" + 0.022*"senate" + 0.018*"biden" + 0.018*"election" + 0.018*"amp" + 0.017*"impeachment" + 0.01

In [33]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [34]:
 # Let's Visualize
pyLDAvis.display(lda_display)